In [19]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) 
# para poder visualizar todas las columnas de los DataFrames


# Ignorar warnings
# ==============================================================================
import warnings
warnings.filterwarnings("ignore")


# Para gestión de fechas
# -----------------------------------------------------------------------
from datetime import datetime


# Para guardar DataFrames en Excel
# -----------------------------------------------------------------------
from pandas import ExcelWriter


# Para generar todas las posibles combinaciones
# -----------------------------------------------------------------------
import itertools


# Importar nuestras funciones
# -----------------------------------------------------------------------
from src import soporte_eda as se


In [20]:
# Cargar el DataFrame consolidado desde la Fase 1
df_consolidado = pd.read_csv("datos_consolidados.csv", sep=';', encoding='utf-8')


In [21]:
# Identificar y manejar valores nulos
print("\nReporte inicial de valores nulos:")
reporte_nulos = se.generar_reporte_nulos(df_consolidado)
print(reporte_nulos)


Reporte inicial de valores nulos:
                            Numero_nulos  Porcentaje_nulos Tipo_dato
CÓDIGO ÓRGÃO SUPERIOR                  0              0.00     int64
NOMBRE ÓRGANO SUPERIOR                 0              0.00    object
CÓDIGO ÓRGANO                          0              0.00     int64
NOMBRE ÓRGANO                          0              0.00    object
CÓDIGO UNIDAD GESTORA                  0              0.00     int64
NOMBRE UNIDAD GESTORA                  0              0.00    object
CATEGORÍA ECONÓMICA                    0              0.00    object
ORIGEN INGRESO                         0              0.00    object
TIPO INGRESO                           0              0.00    object
DETALLE                                0              0.00    object
VALOR PREVISTO ACTUALIZADO             0              0.00   float64
VALOR LANZADO                          0              0.00   float64
VALOR REALIZADO                        0              0.00   float64

In [22]:
df_consolidado['PORCENTAJE REALIZADO'] = df_consolidado['PORCENTAJE REALIZADO'].fillna(0)
df_consolidado['VALOR PREVISTO ACTUALIZADO'] = df_consolidado['VALOR PREVISTO ACTUALIZADO'].fillna(0)
df_consolidado['VALOR LANZADO'] = df_consolidado['VALOR LANZADO'].fillna(0)
df_consolidado['VALOR REALIZADO'] = df_consolidado['VALOR REALIZADO'].fillna(0)
df_consolidado['CATEGORÍA ECONÓMICA'] = df_consolidado['CATEGORÍA ECONÓMICA'].fillna("DESCONOCIDO")
df_consolidado['ORIGEN INGRESO'] = df_consolidado['ORIGEN INGRESO'].fillna("DESCONOCIDO")
df_consolidado['TIPO INGRESO'] = df_consolidado['TIPO INGRESO'].fillna("DESCONOCIDO")
df_consolidado['DETALLE'] = df_consolidado['DETALLE'].fillna("DESCONOCIDO")
df_consolidado['AÑO EJERCICIO'] = df_consolidado['AÑO EJERCICIO'].fillna(np.nan)

In [23]:
# Estandarizar formatos de fechas
df_consolidado['FECHA LANZAMIENTO'] = pd.to_datetime(df_consolidado['FECHA LANZAMIENTO'], errors='coerce')
print("Formato de fechas estandarizado.")

Formato de fechas estandarizado.


In [24]:
# Verificar fechas ausentes en el rango
fechas_ausentes = se.rango_fechas(df_consolidado, 'FECHA LANZAMIENTO')
print(f"Fechas ausentes: {fechas_ausentes}")

Fechas ausentes: DatetimeIndex(['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04',
               '2014-01-05', '2014-01-06', '2014-01-07', '2014-01-08',
               '2014-01-09', '2014-01-10',
               ...
               '2016-01-02', '2016-01-03', '2016-01-09', '2016-01-17',
               '2016-01-24', '2016-05-08', '2020-03-29', '2020-12-25',
               '2020-12-27', '2020-12-31'],
              dtype='datetime64[ns]', length=738, freq=None)


In [25]:
# Verificar y eliminar duplicados
duplicados = df_consolidado.duplicated().sum()
print(f"Filas duplicadas encontradas: {duplicados}")

if duplicados > 0:
    df_consolidado = df_consolidado.drop_duplicates()
    print(f"Filas duplicadas eliminadas. Total de filas ahora: {len(df_consolidado)}")


Filas duplicadas encontradas: 0


In [26]:
# Corregir inconsistencias en categorías económicas
# Convertir texto a minúsculas y eliminar espacios adicionales
df_consolidado['CATEGORÍA ECONÓMICA'] = df_consolidado['CATEGORÍA ECONÓMICA'].str.strip().str.lower()
df_consolidado['ORIGEN INGRESO'] = df_consolidado['ORIGEN INGRESO'].str.strip().str.lower()
df_consolidado['TIPO INGRESO'] = df_consolidado['TIPO INGRESO'].str.strip().str.lower()
df_consolidado['DETALLE'] = df_consolidado['DETALLE'].str.strip().str.lower()

In [27]:
# Verificar resultados finales de la limpieza
print("\nInformación final del DataFrame limpio:")
print(df_consolidado.info())
print(df_consolidado.head())


Información final del DataFrame limpio:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026299 entries, 0 to 1026298
Data columns (total 16 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   CÓDIGO ÓRGÃO SUPERIOR       1026299 non-null  int64         
 1   NOMBRE ÓRGANO SUPERIOR      1026299 non-null  object        
 2   CÓDIGO ÓRGANO               1026299 non-null  int64         
 3   NOMBRE ÓRGANO               1026299 non-null  object        
 4   CÓDIGO UNIDAD GESTORA       1026299 non-null  int64         
 5   NOMBRE UNIDAD GESTORA       1026299 non-null  object        
 6   CATEGORÍA ECONÓMICA         1026299 non-null  object        
 7   ORIGEN INGRESO              1026299 non-null  object        
 8   TIPO INGRESO                1026299 non-null  object        
 9   DETALLE                     1026299 non-null  object        
 10  VALOR PREVISTO ACTUALIZADO  1026299 non-null  flo